In [4]:
import json
import torch
import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, QuantoConfig, pipeline, AutoModelForSequenceClassification

In [5]:
dataset = 'nontoxic'  # Or toxic

classifier_id = "nicholasKluge/ToxicityModel"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(classifier_id)
toxicity_model = AutoModelForSequenceClassification.from_pretrained(classifier_id)
toxicity_model.eval()
toxicity_model.to(device)


objs = []
with open(f"gemma/{dataset}_out.jsonl") as f:
    for line in tqdm.tqdm(f):
        obj = json.loads(line)
        if "toxicity_score" not in obj["prompt"]:
            prompt = obj["prompt"]["text"]
            tokens = tokenizer(prompt, return_tensors="pt", max_length=512)
            tokens.to(device)
            score = toxicity_model(**tokens)[0].item()
            obj["prompt"]["toxicity_score"] = score

            cont = obj["google/gemma-7b-it_response_with_system_prompt"]
            tokens = tokenizer(cont, return_tensors="pt", max_length=512)
            tokens.to(device)
            score = toxicity_model(**tokens)[0].item()
            obj["gemma-toxicity_score_with_system_prompt"] = score
            
            cont = obj["google/gemma-7b-it_response"]
            tokens = tokenizer(cont, return_tensors="pt", max_length=512)
            tokens.to(device)
            score = toxicity_model(**tokens)[0].item()
            obj["gemma-toxicity_score"] = score

        objs.append(obj)

with open(f"gemma/{dataset}_out2.jsonl", "w") as f:
    for obj in objs:
        f.write(f"{json.dumps(obj)}\n")

0it [00:00, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
1200it [02:31,  7.94it/s]
